In [1]:
import pandas as pd

# Load the uploaded CSV file
file_path = '../data/raw_ISC_catalog.csv'

# Read the raw data
with open(file_path, 'r') as file:
    raw_data = file.readlines()

# Display the first few lines of the raw data to understand its structure
raw_data[:10]

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xca in position 88: invalid continuation byte

In [2]:
# Attempt to read the file with a different encoding
with open(file_path, 'r', encoding='latin1') as file:
    raw_data = file.readlines()

# Display the first few lines of the raw data to understand its structure
raw_data[:10]

['International Seismological Centre\n',
 'ISC: Event catalogue\n',
 '\n',
 'Any use of data from the ISCÊshould be cited. The correct format for citations may be found on ourÊcitation page.\n',
 '\n',
 '"Once the search has completed, a compressedÊKML fileÊwill be available to view the results inÊGoogle Earth."\n',
 '\n',
 'Make an event map\n',
 '\n',
 'Search summary:\n']

The file appears to contain introductory text and metadata that needs to be cleaned before extracting the relevant earthquake data. I'll clean up the file and extract the data into a structured format.

Let's start by identifying and isolating the actual data lines.

In [6]:
# Identify the start of the actual data by finding the first line with data columns
data_start_index = None
for index, line in enumerate(raw_data):
    if 'Date' in line and 'Time' in line and 'Latitude' in line:
        data_start_index = index
        break

# Extract the header and data lines
header_line = raw_data[data_start_index].strip()
data_lines = raw_data[data_start_index + 1:]

# Clean and split the header and data lines into columns
header = header_line.split(',')
data = [line.strip().split(',') for line in data_lines]

# Create a DataFrame from the extracted data
earthquake_data = pd.DataFrame(data, columns=header)

# Display the first few rows of the cleaned DataFrame
import ace_tools as tools; tools.display_dataframe_to_user(name="Cleaned Earthquake Data", dataframe=earthquake_data)
earthquake_data.head()

TypeError: list indices must be integers or slices, not NoneType

It seems that the code did not find the correct header line, resulting in a NoneType error. I'll adjust the code to better identify and extract the data columns.

Let's reprocess the file and look for the data header line more carefully.

In [7]:
# Identify the start of the actual data by finding the first line with data columns
data_start_index = None
for index, line in enumerate(raw_data):
    if 'Date' in line and 'Time' in line and 'Latitude' in line:
        data_start_index = index
        break

# Check if the header line was found
if data_start_index is not None:
    # Extract the header and data lines
    header_line = raw_data[data_start_index].strip()
    data_lines = raw_data[data_start_index + 1:]

    # Clean and split the header and data lines into columns
    header = header_line.split(',')
    data = [line.strip().split(',') for line in data_lines]

    # Create a DataFrame from the extracted data
    earthquake_data = pd.DataFrame(data, columns=header)
else:
    earthquake_data = pd.DataFrame()

# Display the first few rows of the cleaned DataFrame
import ace_tools as tools; tools.display_dataframe_to_user(name="Cleaned Earthquake Data", dataframe=earthquake_data)
earthquake_data.head()

ModuleNotFoundError: No module named 'ace_tools'

It appears that the data header line wasn't found correctly. Let's manually inspect the raw data to determine the correct approach for extracting the actual data.

Here are the first 50 lines of the raw data for review:

In [8]:
{''.join(raw_data[:50])}

{'International Seismological Centre\nISC: Event catalogue\n\nAny use of data from the ISCÊshould be cited. The correct format for citations may be found on ourÊcitation page.\n\n"Once the search has completed, a compressedÊKML fileÊwill be available to view the results inÊGoogle Earth."\n\nMake an event map\n\nSearch summary:\nDatabase: Reviewed ISC Bulletin\nSearch type: Rectangular search\nLatitude range: 15 to 28\nLongitude range: 90 to 100\nStart date: 1600-05-01 00:00:00\nEnd date: 2022-06-01 00:00:00\nEvents found: 8602\nDate of search: Mon May 27 04:38:07 2024 GMT\n\nAgencies whose data contributed towards the results of this search are listed at the bottom of the page.\n\nPlease note:\n\nData on or after 2022/06/01 have not been reviewed by the ISC.\n\nDATA_TYPE EVENT_CATALOGUE\nReviewed ISC Bulletin\n----EVENT-----|-----------------------ORIGIN (PRIME HYPOCENTRE)----------------|------MAGNITUDES-----...\n"  EVENTID,TYPE,AUTHOR   ,DATE      ,TIME       ,LAT     ,LON      ,DEPT

In [9]:
# Display more lines to understand the structure and find where the actual data starts
raw_data[10:60]

['Database: Reviewed ISC Bulletin\n',
 'Search type: Rectangular search\n',
 'Latitude range: 15 to 28\n',
 'Longitude range: 90 to 100\n',
 'Start date: 1600-05-01 00:00:00\n',
 'End date: 2022-06-01 00:00:00\n',
 'Events found: 8602\n',
 'Date of search: Mon May 27 04:38:07 2024 GMT\n',
 '\n',
 'Agencies whose data contributed towards the results of this search are listed at the bottom of the page.\n',
 '\n',
 'Please note:\n',
 '\n',
 'Data on or after 2022/06/01 have not been reviewed by the ISC.\n',
 '\n',
 'DATA_TYPE EVENT_CATALOGUE\n',
 'Reviewed ISC Bulletin\n',
 '----EVENT-----|-----------------------ORIGIN (PRIME HYPOCENTRE)----------------|------MAGNITUDES-----...\n',
 '"  EVENTID,TYPE,AUTHOR   ,DATE      ,TIME       ,LAT     ,LON      ,DEPTH,DEPFIX,AUTHOR   ,TYPE  ,MAG "\n',
 '" 16957826,ke  ,ISC      ,1904-08-30,11:43:37.59, 25.5166,  97.0211, 15.0,TRUE  ,ABE1     ,mB    , 7.6,AN2      ,Ms    , 6.8,BJI      ,Ms    , 6.8"\n',
 '"   914281,ke  ,GUTE     ,1906-08-31,14:57:30 

The data starts after several lines of metadata and introductory text. The actual data seems to start from the line that begins with "EVENTID,TYPE,AUTHOR,...". This line indicates the column headers, followed by the earthquake event data.

Let's clean and restructure the columns based on this observation.

I'll extract the column headers and the subsequent data lines correctly.

In [10]:
# Identify the correct start index for the actual data
data_start_index = None
for index, line in enumerate(raw_data):
    if 'EVENTID' in line and 'TYPE' in line and 'AUTHOR' in line:
        data_start_index = index
        break

# Extract the header and data lines
if data_start_index is not None:
    header_line = raw_data[data_start_index].strip().replace('"','')
    data_lines = raw_data[data_start_index + 1:]

    # Clean and split the header and data lines into columns
    header = [col.strip() for col in header_line.split(',')]
    data = [line.strip().replace('"','').split(',') for line in data_lines if line.strip()]

    # Create a DataFrame from the extracted data
    earthquake_data = pd.DataFrame(data, columns=header)

    # Display the cleaned DataFrame to the user
    import ace_tools as tools; tools.display_dataframe_to_user(name="Cleaned Earthquake Data", dataframe=earthquake_data)
else:
    earthquake_data = pd.DataFrame()
    
earthquake_data.head()

ValueError: 12 columns passed, passed data had 123 columns

The data lines have an inconsistent number of columns, possibly due to the presence of commas within the values or additional formatting issues.

To handle this, I'll adjust the code to better parse the data lines by accounting for possible issues like extra commas within values.

Let's reprocess the data with improved parsing logic. ​

In [11]:
import csv

# Identify the correct start index for the actual data
data_start_index = None
for index, line in enumerate(raw_data):
    if 'EVENTID' in line and 'TYPE' in line and 'AUTHOR' in line:
        data_start_index = index
        break

# Extract the header and data lines
if data_start_index is not None:
    header_line = raw_data[data_start_index].strip().replace('"','')
    data_lines = raw_data[data_start_index + 1:]

    # Clean and split the header and data lines into columns
    header = [col.strip() for col in header_line.split(',')]
    data = []
    
    for line in data_lines:
        if line.strip():
            reader = csv.reader([line.strip().replace('"','')])
            data.extend(list(reader))
    
    # Create a DataFrame from the extracted data
    earthquake_data = pd.DataFrame(data, columns=header)
    
    # Display the cleaned DataFrame to the user
    import ace_tools as tools; tools.display_dataframe_to_user(name="Cleaned Earthquake Data", dataframe=earthquake_data)
else:
    earthquake_data = pd.DataFrame()

earthquake_data.head()

ValueError: 12 columns passed, passed data had 123 columns

It appears there are still inconsistencies in the data, likely due to varying numbers of columns in different rows. I will take a more manual approach to identify and parse rows with the correct number of columns.

Let's inspect the rows manually to find out where the inconsistencies lie and adjust accordingly. ​

In [12]:
# Manually inspect a few rows to understand the inconsistencies
for line in data_lines[:20]:
    print(len(line.strip().replace('"','').split(',')), line.strip().replace('"','').split(','))

18 [' 16957826', 'ke  ', 'ISC      ', '1904-08-30', '11:43:37.59', ' 25.5166', '  97.0211', ' 15.0', 'TRUE  ', 'ABE1     ', 'mB    ', ' 7.6', 'AN2      ', 'Ms    ', ' 6.8', 'BJI      ', 'Ms    ', ' 6.8']
21 ['   914281', 'ke  ', 'GUTE     ', '1906-08-31', '14:57:30   ', ' 27.0000', '  97.0000', '100.0', '      ', 'PAS      ', 'MS    ', ' 7.0', 'ABE1     ', 'mB    ', ' 6.7', 'GUTE     ', 'UK    ', ' 7.0', 'B&D      ', 'UK    ', ' 7.0']
18 [' 16957984', 'ke  ', 'ISC      ', '1908-02-09', '18:13:17.58', ' 29.2781', ' 100.6556', ' 15.0', 'TRUE  ', 'ISC      ', 'MS    ', ' 6.7', 'BJI      ', 'Ms    ', ' 7.1', 'B&D      ', 'UK    ', ' 7.1']
27 [' 16958007', 'ke  ', 'ISC      ', '1908-12-12', '12:54:57.27', ' 26.9477', '  96.7734', ' 15.0', 'TRUE  ', 'ISC      ', 'MS    ', ' 7.0', 'PAS      ', 'MS    ', ' 7.5', 'ABE1     ', 'mB    ', ' 7.6', 'AN2      ', 'Ms    ', ' 7.0', 'BJI      ', 'Ms    ', ' 7.0', 'GUTE     ', 'UK    ', ' 7.5']
27 [' 16958191', 'ke  ', 'ISC      ', '1912-05-23', '02:24:0

The data lines have inconsistent lengths, meaning some rows have more columns than others. To handle this, I will:

Identify and normalize the maximum number of columns found in any row.
Adjust the header and data accordingly to handle these variations.
Let's process the data again, ensuring we handle rows with varying numbers of columns.

In [14]:
# Determine the maximum number of columns in the data
max_columns = max(len(line.strip().replace('"','').split(',')) for line in data_lines)

# Create a new header with the maximum number of columns
header_line = raw_data[data_start_index].strip().replace('"','')
header = [col.strip() for col in header_line.split(',')]
header += [f'Extra_{i}' for i in range(max_columns - len(header))]

# Clean and split the data lines into columns, padding with empty strings for missing values
data = []
for line in data_lines:
    if line.strip():
        reader = csv.reader([line.strip().replace('"','')])
        row = list(reader)[0]
        row += [''] * (max_columns - len(row))  # Pad the row with empty strings if needed
        data.append(row)

# Create a DataFrame from the extracted data
earthquake_data = pd.DataFrame(data, columns=header)

# Display the cleaned DataFrame to the user
# import ace_tools as tools; tools.display_dataframe_to_user(name="Cleaned Earthquake Data", dataframe=earthquake_data)
earthquake_data.head()

,EVENTID,TYPE,AUTHOR,DATE,TIME,LAT,LON,DEPTH,DEPFIX,AUTHOR,...,Extra_101,Extra_102,Extra_103,Extra_104,Extra_105,Extra_106,Extra_107,Extra_108,Extra_109,Extra_110
0,16957826,ke,ISC,1904-08-30,11:43:37.59,25.5166,97.0211,15.0,TRUE,ABE1,...,,,,,,,,,,
1,914281,ke,GUTE,1906-08-31,14:57:30,27.0000,97.0000,100.0,,PAS,...,,,,,,,,,,
2,16957984,ke,ISC,1908-02-09,18:13:17.58,29.2781,100.6556,15.0,TRUE,ISC,...,,,,,,,,,,
3,16958007,ke,ISC,1908-12-12,12:54:57.27,26.9477,96.7734,15.0,TRUE,ISC,...,,,,,,,,,,
4,16958191,ke,ISC,1912-05-23,02:24:04.06,21.0361,96.7366,15.0,TRUE,ISC,...,,,,,,,,,,


In [15]:
import pandas as pd
import csv
from datetime import datetime

# Load the uploaded CSV file
file_path = '../data/raw_ISC_catalog.csv'

# Read the raw data
with open(file_path, 'r', encoding='latin1') as file:
    raw_data = file.readlines()

# Identify the correct start index for the actual data
data_start_index = None
for index, line in enumerate(raw_data):
    if 'EVENTID' in line and 'TYPE' in line and 'AUTHOR' in line:
        data_start_index = index
        break

# Extract the header and data lines
if data_start_index is not None:
    header_line = raw_data[data_start_index].strip().replace('"','')
    data_lines = raw_data[data_start_index + 1:]

    # Clean and split the header and data lines into columns
    header = [col.strip() for col in header_line.split(',')]
    
    # Determine the maximum number of columns in the data
    max_columns = max(len(line.strip().replace('"','').split(',')) for line in data_lines)

    # Create a new header with the maximum number of columns
    header += [f'Extra_{i}' for i in range(max_columns - len(header))]

    # Clean and split the data lines into columns, padding with empty strings for missing values
    data = []
    for line in data_lines:
        if line.strip():
            reader = csv.reader([line.strip().replace('"','')])
            row = list(reader)[0]
            row += [''] * (max_columns - len(row))  # Pad the row with empty strings if needed
            data.append(row)

    # Create a DataFrame from the extracted data
    earthquake_data = pd.DataFrame(data, columns=header)

    # Convert DEPTH and MAG columns to numeric, forcing errors to NaN
    earthquake_data['DEPTH'] = pd.to_numeric(earthquake_data['DEPTH'], errors='coerce')
    earthquake_data['MAG'] = pd.to_numeric(earthquake_data['MAG'], errors='coerce')

    # Drop rows where DEPTH or MAG is NaN
    earthquake_data = earthquake_data.dropna(subset=['DEPTH', 'MAG'])

    # Function to parse dates flexibly
    def parse_date(date_str):
        for fmt in ('%d/%m/%Y', '%d-%m-%Y', '%Y-%m-%d', '%d/%m/%y'):
            try:
                return datetime.strptime(date_str, fmt).strftime('%d-%m-%Y')
            except ValueError:
                continue
        return None

    # Apply the date parsing function to the DATE column
    earthquake_data['DATE'] = earthquake_data['DATE'].apply(parse_date)

    # Drop rows where DATE could not be parsed
    earthquake_data = earthquake_data.dropna(subset=['DATE'])

else:
    earthquake_data = pd.DataFrame()

# Save the cleaned DataFrame to a new CSV file
output_file_path = '../data/Cleaned_ISC_Data.csv'

earthquake_data.to_csv(output_file_path, index=False)

print(f"Cleaned data saved to {output_file_path}")

earthquake_data.head()

Cleaned data saved to ../data/Cleaned_ISC_Data.csv


,EVENTID,TYPE,AUTHOR,DATE,TIME,LAT,LON,DEPTH,DEPFIX,AUTHOR,...,Extra_101,Extra_102,Extra_103,Extra_104,Extra_105,Extra_106,Extra_107,Extra_108,Extra_109,Extra_110
0,16957826,ke,ISC,30-08-1904,11:43:37.59,25.5166,97.0211,15.0,TRUE,ABE1,...,,,,,,,,,,
1,914281,ke,GUTE,31-08-1906,14:57:30,27.0000,97.0000,100.0,,PAS,...,,,,,,,,,,
2,16957984,ke,ISC,09-02-1908,18:13:17.58,29.2781,100.6556,15.0,TRUE,ISC,...,,,,,,,,,,
3,16958007,ke,ISC,12-12-1908,12:54:57.27,26.9477,96.7734,15.0,TRUE,ISC,...,,,,,,,,,,
4,16958191,ke,ISC,23-05-1912,02:24:04.06,21.0361,96.7366,15.0,TRUE,ISC,...,,,,,,,,,,


In [14]:
# # import pandas as pd
# # from datetime import datetime

# # Load the cleaned data file
# file_path = '../data/Cleaned_ISC_Data.csv'
# earthquake_data = pd.read_csv(file_path)

# # Function to parse dates flexibly and convert to DD-MM-YYYY format
# def parse_date(date_str):
#     for fmt in ('%d/%m/%Y', '%d-%m-%Y', '%Y-%m-%d', '%d/%m/%y'):
#         try:
#             return datetime.strptime(date_str, fmt).strftime('%d-%m-%Y')
#         except ValueError:
#             continue
#     return None

# # Apply the date parsing function to the DATE column
# earthquake_data['DATE'] = earthquake_data['DATE'].apply(parse_date)

# # Drop rows where DATE could not be parsed
# earthquake_data = earthquake_data.dropna(subset=['DATE'])

# # Save the cleaned DataFrame with corrected date format to a new CSV file
# output_file_path = '../data/Cleaned_ISC_Data2.csv'
# earthquake_data.to_csv(output_file_path, index=False)

# print(f"Cleaned data saved to {output_file_path}")

# earthquake_data.head()

Cleaned data saved to ../data/Cleaned_ISC_Data2.csv


,EVENTID,TYPE,AUTHOR,DATE,TIME,LAT,LON,DEPTH,DEPFIX,AUTHOR.1,...,Extra_101,Extra_102,Extra_103,Extra_104,Extra_105,Extra_106,Extra_107,Extra_108,Extra_109,Extra_110
0,16957826,ke,ISC,30-08-1904,11:43:37.59,25.5166,97.0211,15.0,TRUE,ABE1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,914281,ke,GUTE,31-08-1906,14:57:30,27.0000,97.0000,100.0,,PAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,16957984,ke,ISC,09-02-1908,18:13:17.58,29.2781,100.6556,15.0,TRUE,ISC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16958007,ke,ISC,12-12-1908,12:54:57.27,26.9477,96.7734,15.0,TRUE,ISC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16958191,ke,ISC,23-05-1912,02:24:04.06,21.0361,96.7366,15.0,TRUE,ISC,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
